# EDA of Steel Defect Detection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train_one_fault.csv', index_col='id')
test = pd.read_csv('data/test.csv', index_col='id')

In [3]:
# check if any columns have missing values
# print(train.isnull().any())
# print(test.isnull().any())

In [4]:
# print all the columns
# print(train.columns)

In [5]:
y_columns = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
y = train[y_columns]
X = train.drop(y_columns, axis=1)

# print the first 5 rows of the dataframe
# print(X.head())
# print(y.head())

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(test)

# undo onehot encoding for y 
y = y.apply(lambda row: np.argmax(row.values), axis=1)

In [7]:
from sklearn.decomposition import PCA

# pca = PCA(n_components=27)
# pca.fit(X)

# eigenvalues = pca.explained_variance_
# eigenvalues

In [8]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.datasets import make_classification

# # Create the RFE object and rank each feature
# svc = SVC(kernel="linear", C=1)
# rfe = RFE(estimator=svc, n_features_to_select=25, step=1)
# rfe = rfe.fit(X, y)

# # Summarize the selection of the attributes
# print(f"Num Features: {rfe.n_features_}")
# print(f"Selected Features: {rfe.support_}")
# print(f"Feature Ranking: {rfe.ranking_}")

In [9]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


# models = {
#     "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
#     "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
#     "MLP Classifier": MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
# }

# # Perform 5-fold cross-validation and calculate AUC-ROC for each model
# cv = StratifiedKFold(n_splits=5)

# for name, model in models.items():
#     # Generate cross-validated estimates for each input data point
#     # Method='predict_proba' is used since we need probabilities to calculate AUC-ROC
#     y_pred = cross_val_predict(model, X, y, cv=cv, method='predict_proba')
    
#     # Calculate the AUC-ROC score
#     auc_roc = roc_auc_score(y, y_pred, multi_class='ovr', average='macro')
    
#     print(f"{name}: AUC-ROC = {auc_roc:.4f}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 14.0.1.dev0+gba5374836.d20240125 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [10]:
# from sklearn.model_selection import GridSearchCV, train_test_split

# # Custom GridSearchCV subclass
# class CustomGridSearchCV(GridSearchCV):
#     def _run_search(self, evaluate_candidates):
#         """Override the method to print parameter settings."""
#         def custom_evaluate_candidates(candidate_params):
#             for parameters in candidate_params:
#                 print(f"Evaluating: {parameters}")
#                 evaluate_candidates([parameters])
                
#         super()._run_search(custom_evaluate_candidates)

# # Multi-class AUC-ROC scorer
# def multiclass_roc_auc_score(y_true, y_pred, average="macro"):
#     y_bin = label_binarize(y_true, classes=np.unique(y_true))
#     temp = roc_auc_score(y_bin, y_pred, average=average, multi_class='ovr')
#     print(temp)
#     return temp

# # Defining the model
# gb = GradientBoostingClassifier(random_state=42)

# # Setting up the parameter grid
# param_grid = {
#     'n_estimators': [100, 400, 700, 1000],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2],
#     'max_depth': [3, 4, 5],
#     'min_samples_split': [2, 4, 6],
#     'min_samples_leaf': [1, 2, 3]
# }
# # Setting up CustomGridSearchCV with AUC-ROC scoring
# auc_roc_scorer = make_scorer(multiclass_roc_auc_score, needs_proba=True)
# grid_search = CustomGridSearchCV(estimator=gb, param_grid=param_grid, cv=5, scoring=auc_roc_scorer, n_jobs=-1)

# # Fitting CustomGridSearchCV
# grid_search.fit(X, y)

# # Best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

In [12]:
rf_model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=3, min_samples_leaf=2, min_samples_split=2, random_state=42)
rf_model.fit(X, y)
y_test = rf_model.predict(X_test)
output = np.zeros((len(y_test), 7))
for i in range(len(y_test)):
    output[i, y_test[i]] = 1
output = pd.DataFrame(output, columns=y_columns, index=test.index)
output.to_csv('submissions/onefault_gbc1.csv')